In [1]:
# Import the required modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import MNIST
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

# Fix the randomness
seed = 1234
torch.manual_seed(seed)

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
HP = {
    "lr": [1e-1,1e-2,1e-3,1e-4],
    "neurons": [1,2,3],
    "hidden_layers": [[200, 100, 50], [200, 100], [200]],
    "activation_funcs": [nn.ReLU(),nn.LeakyReLU(),nn.Sigmoid()],
    "batch_size":32,
    "num_epoch":3
}

In [4]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as T

train_transform = T.Compose ([
T.ToTensor(),
T.Grayscale(),
T.Normalize(mean =(0.5,), std=(0.5,))
])
val_transform = test_transform = T.Compose([
T.ToTensor(),
T.Grayscale(),
T.Normalize(mean =(0.5,) , std=(0.5,))
])

train_set = CIFAR10 ( root ="CIFAR10", train =True ,transform = train_transform , download = True )
test_set = CIFAR10 ( root ="CIFAR10", train =False ,transform = test_transform , download = True )
train_set_length = int(0.8 * len(train_set))
val_set_length = len(train_set) - train_set_length
train_set, val_set = random_split(train_set, [train_set_length, val_set_length])


train_loader = DataLoader(train_set, batch_size=HP["batch_size"], shuffle=True)
test_loader = DataLoader(test_set, batch_size=HP["batch_size"])
val_loader = DataLoader(val_set, batch_size=HP["batch_size"])


Files already downloaded and verified
Files already downloaded and verified


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# Define the ANN
class MyModel(nn.Module):

    def __init__(self, output_shape, hidden_layers,activation):
        super().__init__()
        layers = [32*32*1] + hidden_layers + [output_shape]
        modules = []
        
        for i in range(len(layers)-1):
            modules.append(nn.Linear(layers[i], layers[i+1]))
            modules.append(activation)
        self.lin = nn.Sequential(*modules)

    def forward(self, x):
        x = torch.flatten(x, 1)
        return self.lin(x)

In [18]:
def fit(m,i,j,k):
    for epoch in tqdm(range(HP["num_epoch"])):
        # Training
        model.train()
        accum_train_loss = 0
        for n, (imgs, labels) in enumerate(train_loader, start=1):
            imgs, labels = imgs.to(device), labels.to(device)
            output = model(imgs)
            loss = loss_function(output, labels)

            # accumlate the loss
            accum_train_loss += loss.item()

            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        accum_val_loss = 0
        with torch.no_grad():
            for x, (imgs, labels) in enumerate(val_loader, start=1):
                imgs, labels = imgs.to(device), labels.to(device)
                output = model(imgs)
                accum_val_loss += loss_function(output, labels).item()
        
        # print statistics of the epoch
        print(f'Epoch = {epoch} | Train Loss = {accum_train_loss / n:.4f}\tVal Loss = {accum_val_loss / x:.4f}')

    return [HP["neurons"][m],HP["hidden_layers"][i],HP["lr"][j],HP["activation_funcs"][k],accum_train_loss / n ,accum_val_loss / x]

In [19]:
results = []
for m in range(len(HP["neurons"])):
    for i in range(len(HP["hidden_layers"])):
        for j in range(len(HP["lr"])):
            for k in range(len(HP["activation_funcs"])):
                
                model = MyModel(10, hidden_layers=((np.array(HP["hidden_layers"][i])*HP["neurons"][m]).tolist()),activation=HP["activation_funcs"][k]).to(device)
                loss_function = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=HP["lr"][j])
                a = fit(m,i,j,k)
                print(a)
                results.append(a)            
                

 33%|███▎      | 1/3 [00:15<00:31, 15.90s/it]

Epoch = 0 | Train Loss = 2.8219	Val Loss = 2.3026


 67%|██████▋   | 2/3 [00:31<00:15, 15.89s/it]

Epoch = 1 | Train Loss = 2.3026	Val Loss = 2.3026


100%|██████████| 3/3 [00:47<00:00, 15.89s/it]

Epoch = 2 | Train Loss = 2.3026	Val Loss = 2.3026
[1, [200, 100, 50], 0.1, ReLU(), 2.3025848865509033, 2.3025848880743447]


TypeError: append() takes exactly one argument (0 given)